In [1]:
import os, sys
import h5py
import numpy as np
from scipy.io import loadmat
import cv2
import matplotlib
%matplotlib inline 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from numpy import matrix as mat

In [2]:
m = loadmat("valid.mat")
print(m.keys())

dict_keys(['__header__', '__version__', '__globals__', 'annot'])


In [3]:
image_name = m["annot"][0][0][0]
print(image_name[0])
center = m["annot"][0][0][1]
print(center[:].shape)
scale = m["annot"][0][0][2]
print(scale.shape)
gt = m["annot"][0][0][3]
print(gt.shape)
lenth = ((gt[0][0][0]-gt[0][0][8])**2+(gt[0][1][0]-gt[0][1][8])**2+
             (gt[0][2][0]-gt[0][2][8])**2)**(0.5)
scale = lenth/1000
print(scale)

[array(['S9_Posing_1.55011271_000001'], dtype='<U27')]
(394, 2)
(394, 1)
(394, 3, 17)
0.5064847669327615


In [4]:
kp = np.zeros((18,394,3))
for i in range(17):
    for frame in range(394):
        kp[i][frame][0] = gt[frame][0][i]
        kp[i][frame][1] = gt[frame][1][i]
        kp[i][frame][2] = gt[frame][2][i]
print(kp[0][:10])   # [ ith point ][ frame ][ x,y,z ]
# 50Hz video，sample 5 --> 0.1 second per image 

[[-3.21855365e+01 -5.43171724e+02  5.40351766e+03]
 [-2.96551493e+01 -5.43688575e+02  5.40388307e+03]
 [-2.47249452e+01 -5.46610386e+02  5.40795585e+03]
 [-1.30441613e+01 -5.52146899e+02  5.41697698e+03]
 [-3.50979292e+00 -5.53386275e+02  5.42395218e+03]
 [ 8.59488443e+00 -5.46680507e+02  5.42241348e+03]
 [ 3.61400074e+01 -5.38359949e+02  5.40629071e+03]
 [ 6.72255834e+01 -5.38629266e+02  5.39471965e+03]
 [ 8.90948864e+01 -5.36624251e+02  5.38774592e+03]
 [ 9.77728569e+01 -5.37230692e+02  5.38483065e+03]]


In [5]:
for frame in range(394):
    point_sum = 0
    for p in range(17):
        point_sum += kp[p][frame]
    kp[17][frame] = point_sum / 17
print(kp[17])

[[ -16.68807687 -666.91435408 5405.8121701 ]
 [ -15.91177218 -666.58006381 5406.4677378 ]
 [ -16.43919606 -668.57042707 5408.01079269]
 ...
 [-256.17816287 -540.606673   5139.26774503]
 [-254.4438796  -521.38684393 5150.10763805]
 [-249.03792971 -504.75943427 5166.64538453]]


In [457]:
point_num = 17
init = 200   # init frame
#time range
t_range = np.arange(0, 5, 0.1)   # t <= 39.4
t_len = len(t_range)
#camera coordinate
x_cam, y_cam, z_cam = 0,0,0
r_cam = 50 # mm
omg_cam = 2 * np.pi
#camera trajectory
px_cam = x_cam + r_cam * np.cos(omg_cam * t_range + np.pi)
py_cam = y_cam + r_cam * np.sin(omg_cam * t_range + np.pi)
pz_cam = z_cam + np.zeros(t_len)
#human trajectory
px_hum= np.zeros((t_len))
py_hum = np.zeros((t_len))
pz_hum = np.zeros((t_len))
for i in range(t_len):
    px_hum[i] = kp[point_num][init+i][0]
    py_hum[i] = kp[point_num][init+i][1]
    pz_hum[i]= kp[point_num][init+i][2]
#print(px_hum[0:10])
#diff trajectory
px_d = px_hum - px_cam
py_d = py_hum - py_cam
pz_d = pz_hum - py_cam
px_d = px_d / scale
py_d = py_d / scale
pz_d = pz_d / scale

In [458]:
#motion calculation
cam_motion = np.zeros((t_len-1,3))
diff_motion = np.zeros((t_len-1,3))
for i in range(t_len-1):
    m_cam_k = np.zeros((1,3))
    m_diff_k = np.zeros((1,3))
    m_cam_k[0][0] = (px_cam[i+1] - px_cam[i]) / 0.1
    m_cam_k[0][1] = (py_cam[i+1] - py_cam[i]) / 0.1
    m_cam_k[0][2] = (pz_cam[i+1] - pz_cam[i]) / 0.1
    m_diff_k[0][0] = (px_d[i+1] - px_d[i]) / 0.1
    m_diff_k[0][1] = (py_d[i+1] - py_d[i]) / 0.1
    m_diff_k[0][2] = (pz_d[i+1] - pz_d[i]) / 0.1
    cam_motion[i] = m_cam_k
    diff_motion[i] = m_diff_k
mc_mean = cam_motion.mean(axis = 0)
md_mean = diff_motion.mean(axis = 0)

In [459]:
def cov_cal(motion1, motion2, motion1_mean, motion2_mean):
    matrix_sum = np.zeros((3,3))
    for i in range(t_len-1):
        matrix_sum += (motion1[i] - motion1_mean).T * (motion2[i] - motion2_mean)
    result = matrix_sum/(t_len-2)
    return result
    
#correlation calculation
cov_md_mc = cov_cal(diff_motion, cam_motion, md_mean, mc_mean)
cov_mc_mc = cov_cal(cam_motion, cam_motion, mc_mean, mc_mean)

In [460]:
#optimal scale S
A = cov_md_mc * cov_mc_mc
B = cov_md_mc * cov_md_mc
C = cov_mc_mc * cov_mc_mc
S = -A.sum()/B.sum()
#solution
print('Scale =', S)
f = B.sum() * S * S + 2 * A.sum() * S + C.sum()
print('F(S) =', f/1000000)
print('Cov_MdMc =', B.sum()/1000000000)
#print('Cov_McMc =', C.sum()/1000000000)

Scale = 0.4727419796091937
F(S) = 38.82206381661224
Cov_MdMc = 63.57091415360486


#角速度重要性大于半径